In [9]:
import os
from PIL import Image
import numpy as np
#import tensorflow as tf
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch
import wandb
wandb.login(key='e595ff5b95c353a42c4bd1f35b70856d4309ef00')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# data directory
data_dir = '/kaggle/input/inaturalist12k/Data/inaturalist_12K'
def get_transforms(data_augmentation):
    if data_augmentation=="yes":
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=30),
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
            transforms.Resize((224,224)),
            transforms.ToTensor()
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor()
        ])
    return transform

transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
train_dataset=torchvision.datasets.ImageFolder(root=data_dir+'/train',transform=get_transforms("no"))
test_dataset=torchvision.datasets.ImageFolder(root=data_dir+'/val',transform=transform)
val_size = int(len(train_dataset) * 0.2)
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=16,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=16,shuffle=False)
#--------------------------------------------------------------------------------------------------------

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets # added for googlenet
#from torchvision.models import GoogLeNet
import torchvision.models as models


# class implementation for googlenet 
class MyGoogLeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(MyGoogLeNet, self).__init__()
        self.googlenet = models.googlenet(pretrained=True)
        self.googlenet.fc = nn.Linear(self.googlenet.fc.in_features, num_classes)

        # Freeze all layers except the last fully connected layer
        for param in self.googlenet.parameters():
            param.requires_grad = False
        for param in self.googlenet.fc.parameters():
            param.requires_grad = True
            
    #forward pass is implemented      
    def forward(self, x):
        x = self.googlenet(x)
        return x

sweep_config = {
    'method': 'random', #grid, random,#bayes
    'name' : 'Random_sweep_cross_entropy',
    'metric': {
      'name': 'valid accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'optimizerr': {
            'values': ["sgd"]
        },
        'learn_rate':{
            'values':["0.001"]
        },
         'momentumm':{
            'values':[".9"]
        },     
        
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='googlenet')
from types import SimpleNamespace
def main():
    with wandb.init() as run:
        params={}
        params=dict(wandb.config)
        params=SimpleNamespace(**params)
        run_name="goolenet training for inaturallist dataset"
        # Initialize the model
        model = MyGoogLeNet().to(device)

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)



        # Train the model
        num_epochs = 10
        for epoch in range(num_epochs):
            model.train()
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.to(device), target.to(device)# Move data and target to the device
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

            # Evaluate the model on train_loader
            model.eval()
            train_correct = 0
            train_total = 0
            train_loss = 0.0
            with torch.no_grad():
                for data, target in train_loader:
                    data, target = data.to(device), target.to(device)# Move data and target to the device
                    output = model(data)
                    loss = criterion(output, target)
                    train_loss += loss.item()
                    _, predicted = torch.max(output.data, 1)
                    train_total += target.size(0)
                    train_correct += (predicted == target).sum().item()

            train_accuracy = 100 * train_correct/train_total
            train_loss /= len(train_loader)

            # Set model to evaluation mode
            model.eval()
            val_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for batch_idx, (data, target) in enumerate(val_loader):
                    data, target = data.to(device), target.to(device)# Move data and target to the device
                    output = model(data)
                    loss = criterion(output, target)
                    val_loss += loss.item()
                    _, predicted = torch.max(output.data, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()

            # Calculate validation metrics
            val_loss /= len(val_loader)
            val_accuracy = 100 * correct / total

            # Print metrics for current epoch
            #print('Epoch: {} \t Training Loss: {:.6f}
            print("epoch",epoch)
            print('Training Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(train_loss, train_accuracy))
            print('Validation Loss: {:.6f} \t Validation Accuracy: {:.6f}'.format(val_loss, val_accuracy))
            wandb.log({'train loss':train_loss,'train accuracy':train_accuracy,'valid loss':val_loss,'valid accuracy':val_accuracy})

wandb.agent(sweep_id, function=main,count=1)



Create sweep with ID: voo0n295
Sweep URL: https://wandb.ai/cs22m010/googlenet/sweeps/voo0n295


wandb: Agent Starting Run: tk38x23u with config:
wandb: 	learn_rate: 0.001
wandb: 	momentumm: .9
wandb: 	optimizerr: sgd


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch 0
Training Loss: 1.190815 	 Training Accuracy: 65.950000
Validation Loss: 1.217925 	 Validation Accuracy: 65.082541
epoch 1
Training Loss: 1.029540 	 Training Accuracy: 69.075000
Validation Loss: 1.071627 	 Validation Accuracy: 66.333167
epoch 2
Training Loss: 0.967437 	 Training Accuracy: 69.700000
Validation Loss: 1.022048 	 Validation Accuracy: 67.033517
epoch 3
Training Loss: 0.925018 	 Training Accuracy: 70.075000
Validation Loss: 0.996571 	 Validation Accuracy: 68.234117
epoch 4
Training Loss: 0.913335 	 Training Accuracy: 70.400000
Validation Loss: 0.995556 	 Validation Accuracy: 67.683842
epoch 5
Training Loss: 0.874065 	 Training Accuracy: 71.300000
Validation Loss: 0.964023 	 Validation Accuracy: 69.384692
epoch 6
Training Loss: 0.862811 	 Training Accuracy: 71.825000
Validation Loss: 0.957113 	 Validation Accuracy: 68.734367
epoch 7
Training Loss: 0.856424 	 Training Accuracy: 71.875000
Validation Loss: 0.952825 	 Validation Accuracy: 69.184592
epoch 8
Training Loss: 0

train accuracy,▁▄▅▅▅▆▇▇▇█
train loss,█▅▄▃▃▂▂▂▂▁
valid accuracy,▁▃▄▆▅▇▆▇▆█
valid loss,█▄▃▃▃▂▂▂▂▁
train accuracy,73.25
train loss,0.81851
valid accuracy,69.93497
valid loss,0.92999
